# Strands Agents with AgentCore Memory (Long term memory) - MemoryManager

## Introduction

This tutorial demonstrates how to build an **intelligent customer support agent** using Strands agents integrated with AgentCore Memory via hooks using **MemoryManager** and **MemorySessionManager**. We'll focus on Long term memory for customer interaction history, remembering purchase details, and provides personalized support based on previous conversations and user preferences.

**NOTE: This is sample use MemoryManager & MemorySessionManager for implementation.**

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Long term Conversational                                                         |
| Agent type          | Customer Support                                                                 |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                      |
| Tutorial components | AgentCore Semantic and User Preferences Memory Extraction, Hooks for storing and retrieving Memory              |
| Example complexity  | Intermediate                                                                     |

You'll learn to:
- Set up AgentCore Memory with Long term strategies using MemoryManager
- Create memory hooks for automatic storage and retrieval with MemorySessionManager
- Build a customer support agent with persistent memory
- Handle customer issues with context from previous interactions

### Scenario Context
In this example, we will build a **Customer Support Use Case**. The agent will remember customer context, including order history, preferences, and previous issues, enabling more personalized and effective support. Conversations with customers are automatically stored using memory hooks, ensuring that important details are never lost. By employing multiple memory strategies such as semantic, and User Preference — the agent can capture a wide range of relevant information. This setup allows the agent to resolve issues with full awareness of the customer's history and preferences. Additionally, the agent is integrated with web search capabilities, making it easy to provide up-to-date product information and troubleshooting guidance as needed.

## Architecture

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## Prerequisites

To execute this tutorial you will need:
- Python 3.10+
- AWS credentials with Amazon Bedrock AgentCore Memory permissions
- Amazon Bedrock AgentCore SDK with MemoryManager support

## Step 1: Install Dependencies and Setup
Let's begin importing all the necessary libraries and defining the clients to make this notebook work.

In [1]:
!pip install -qr requirements.txt

In [2]:
import logging
import json
from typing import Dict, List
from datetime import datetime
from botocore.exceptions import ClientError

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("customer-support")

# Import required modules for Strands Agent
from strands import Agent, tool
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent
from ddgs import DDGS

# Import memory management modules
from bedrock_agentcore_starter_toolkit.operations.memory.manager import Memory, MemoryManager
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies import (
    SemanticStrategy, SummaryStrategy, CustomSemanticStrategy, CustomSummaryStrategy, CustomUserPreferenceStrategy,
    ExtractionConfig, ConsolidationConfig
)
from bedrock_agentcore.memory.constants import BlobMessage, ConversationalMessage, MessageRole, RetrievalConfig
from bedrock_agentcore.memory.session import Actor, MemorySession, MemorySessionManager
from bedrock_agentcore.memory.models import MemoryRecord

# Define message role constants
USER = MessageRole.USER
ASSISTANT = MessageRole.ASSISTANT

In [3]:
# Configuration - Replace with the correct values
REGION = "us-east-1"  
CUSTOMER_ID = "customer_001"
SESSION_ID = f"support_{datetime.now().strftime('%Y%m%d%H%M%S')}"

# Import boto3 for IAM role creation
import boto3
import json as json_module
from botocore.exceptions import ClientError

## Step 1.1: Create IAM Role for Custom Memory Strategies

Custom memory strategies require an execution role that allows AgentCore Memory to invoke Bedrock models for extraction and consolidation. This role is required when using `CustomSemanticStrategy` or `CustomUserPreferenceStrategy`.

In [ ]:
# Create IAM role for AgentCore Memory custom strategies
def create_memory_execution_role():
    """Create IAM role for AgentCore Memory custom strategies with required permissions"""
    iam_client = boto3.client('iam', region_name=REGION)
    
    # Get current AWS account ID
    sts_client = boto3.client('sts', region_name=REGION)
    account_id = sts_client.get_caller_identity()['Account']
    
    role_name = "AgentCoreMemoryExecutionRole"
    role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
    
    # Trust policy for AgentCore Memory service
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "",
                "Effect": "Allow",
                "Principal": {
                    "Service": [
                        "bedrock-agentcore.amazonaws.com"
                    ]
                },
                "Action": "sts:AssumeRole",
                "Condition": {
                    "StringEquals": {
                        "aws:SourceAccount": account_id
                    },
                    "ArnLike": {
                        "aws:SourceArn": f"arn:aws:bedrock-agentcore:{REGION}:{account_id}:*"
                    }
                }
            }
        ]
    }
    
    # Permissions policy for Bedrock model invocation
    permissions_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                    "bedrock:InvokeModelWithResponseStream"
                ],
                "Resource": [
                    "arn:aws:bedrock:*::foundation-model/*",
                    "arn:aws:bedrock:*:*:inference-profile/*"
                ],
                "Condition": {
                    "StringEquals": {
                        "aws:ResourceAccount": account_id
                    }
                }
            }
        ]
    }
    
    try:
        # Check if role already exists
        try:
            existing_role = iam_client.get_role(RoleName=role_name)
            logger.info(f"✅ IAM role already exists: {role_arn}")
            return role_arn
        except ClientError as e:
            if e.response['Error']['Code'] != 'NoSuchEntity':
                raise
        
        # Create the role
        logger.info(f"Creating IAM role: {role_name}")
        iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json_module.dumps(trust_policy),
            Description="Execution role for AgentCore Memory custom strategies",
            Tags=[
                {
                    'Key': 'Purpose',
                    'Value': 'AgentCoreMemory'
                }
            ]
        )
        
        # Attach the permissions policy
        policy_name = "AgentCoreMemoryBedrockAccess"
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=policy_name,
            PolicyDocument=json_module.dumps(permissions_policy)
        )
        
        logger.info(f"✅ Successfully created IAM role: {role_arn}")
        logger.info(f"   - Trust policy: AgentCore Memory service can assume this role")
        logger.info(f"   - Permissions: bedrock:InvokeModel and bedrock:InvokeModelWithResponseStream")
        
        return role_arn
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'AccessDenied':
            logger.error("❌ Access denied creating IAM role. Please ensure you have IAM permissions:")
            logger.error("   - iam:CreateRole")
            logger.error("   - iam:PutRolePolicy")
            logger.error("   - iam:GetRole")
        else:
            logger.error(f"❌ Failed to create IAM role: {e}")
        raise
    except Exception as e:
        logger.error(f"❌ Unexpected error creating IAM role: {e}")
        raise

# Create the execution role
try:
    MEMORY_EXECUTION_ROLE_ARN = create_memory_execution_role()
    logger.info(f"✅ Memory execution role ready: {MEMORY_EXECUTION_ROLE_ARN}")
except Exception as e:
    logger.error(f"❌ Failed to create memory execution role: {e}")
    # For demo purposes, you can manually set the role ARN if creation fails
    # MEMORY_EXECUTION_ROLE_ARN = "arn:aws:iam::YOUR_ACCOUNT_ID:role/YourExistingRole"
    raise

## Step 2: Create Memory Resource for Customer Support

For customer support, we'll use multiple memory strategies:
- **CustomUserPreferenceStrategy**: Captures customer preferences and behavior
- **CustomSemanticStrategy**: Stores order facts and product information

**IMPORTANT**: Custom strategies require an IAM execution role that allows AgentCore Memory to invoke Bedrock models. We created this role in Step 1.1 above.

In [ ]:
# Initialize Memory Manager 
memory_manager = MemoryManager(region_name=REGION)
memory_name = "CustomerSupportLongTermMemory"

# Test basic memory manager initialization and connection
logger.info(f"✅ MemoryManager initialized for region: {REGION}")
logger.info(f"Memory manager type: {type(memory_manager)}")

# Define memory strategies using typed classes
strategies = [
    CustomUserPreferenceStrategy(
        name="CustomerPreferences",
        description="Captures customer preferences and behavior",
        extraction_config=ExtractionConfig(
            append_to_prompt="Extract customer preferences and behavior patterns",
            model_id="anthropic.claude-3-sonnet-20240229-v1:0"
        ),
        consolidation_config=ConsolidationConfig(
            append_to_prompt="Consolidate customer preferences",
            model_id="anthropic.claude-3-sonnet-20240229-v1:0"
        ),
        namespaces=["support/customer/{actorId}/preferences"]
    ),
    CustomSemanticStrategy(
        name="CustomerSupportSemantic",
        description="Stores facts from conversations",
        extraction_config=ExtractionConfig(
            append_to_prompt="Extract factual information from customer support conversations",
            model_id="anthropic.claude-3-sonnet-20240229-v1:0"
        ),
        consolidation_config=ConsolidationConfig(
            append_to_prompt="Consolidate semantic insights from support interactions",
            model_id="anthropic.claude-3-sonnet-20240229-v1:0"
        ),
        namespaces=["support/customer/{actorId}/semantic"]
    )
]

# Validate strategy configuration
logger.info(f"✅ Configured {len(strategies)} memory strategies:")
for i, strategy in enumerate(strategies, 1):
    logger.info(f"  {i}. {strategy.name} ({type(strategy).__name__})")
    logger.info(f"     Description: {strategy.description}")
    logger.info(f"     Namespaces: {strategy.namespaces}")
    if hasattr(strategy, 'extraction_config') and strategy.extraction_config:
        logger.info(f"     Extraction Model: {strategy.extraction_config.model_id}")
    if hasattr(strategy, 'consolidation_config') and strategy.consolidation_config:
        logger.info(f"     Consolidation Model: {strategy.consolidation_config.model_id}"),

# Create memory resource using MemoryManager
logger.info(f"Creating memory '{memory_name}' with {len(strategies)} strategies...")

try:
    memory = memory_manager.get_or_create_memory(
        name=memory_name,
        strategies=strategies,         # Pass typed strategy objects
        description="Memory for customer support agent",
        event_expiry_days=90,          # Memories expire after 90 days
        memory_execution_role_arn=MEMORY_EXECUTION_ROLE_ARN,  # Required for Custom strategies
    )
    memory_id = memory.id
    logger.info(f"✅ Successfully created/retrieved memory with MemoryManager:")
    logger.info(f"   Memory ID: {memory_id}")
    logger.info(f"   Memory Name: {memory.name}")
    logger.info(f"   Memory Status: {memory.status}")
    
except Exception as e:
    # Handle any errors during memory creation with enhanced error reporting
    logger.error(f"❌ Memory creation failed: {e}")
    logger.error(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    
    # Cleanup on error - delete the memory if it was partially created
    if 'memory_id' in locals():
        try:
            logger.info(f"Attempting cleanup of partially created memory: {memory_id}")
            memory_manager.delete_memory(memory_id)
            logger.info(f"✅ Successfully cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"❌ Failed to clean up memory: {cleanup_error}")
    
    # Re-raise the original exception
    raise

In [ ]:
# Test memory manager basic functionality
try:
    # Test listing existing memories
    existing_memories = memory_manager.list_memories()
    logger.info(f"✅ Memory manager connection successful. Found {len(existing_memories)} existing memories")
    
    # Clean up any existing test memories
    for mem in existing_memories:
        if mem.name and mem.name.startswith("CustomerSupportLongTermMemory"):
            logger.info(f"Cleaning up existing test memory: {mem.id}")
            memory_manager.delete_memory(mem.id)
            
except Exception as e:
    logger.error(f"❌ Memory manager test failed: {e}")
    raise

Let's confirm if our memory contains the strategies we assigned

In [ ]:
# Display memory information using MemoryManager
print(f"Memory ID: {memory.id}")
print(f"Memory Name: {memory.name}")
print(f"Memory Description: {memory.description}")
print(f"Memory Status: {memory.status}")
print(f"Number of strategies: {len(strategies)}")
for i, strategy in enumerate(strategies, 1):
    print(f"  {i}. {strategy.name}: {strategy.description}")

## Step 3: Create Agent Tools

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def web_search(query: str, max_results: int = 3) -> str:
    """Search the web for product information, troubleshooting guides, or support articles.
    
    Args:
        query: Search query for product info or troubleshooting
        max_results: Maximum number of results to return
    
    Returns:
        Search results with titles and snippets
    """
    try:
        results = DDGS().text(query, region="us-en", max_results=max_results)
        if not results:
            return "No search results found."
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(f"{i}. {result.get('title', 'No title')}\n   {result.get('body', 'No description')}")
        
        return "\n".join(formatted_results)
    except RatelimitException:
        return "Rate limit reached: Please try again after a short delay."
    except DDGSException as d:
        return f"Search Error: {d}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

@tool
def check_order_status(order_number: str) -> str:
    """Check the status of a customer order.
    
    Args:
        order_number: The order number to check
    
    Returns:
        Order status information
    """
    # Simulate order lookup
    mock_orders = {
        "123456": "iPhone 15 Pro - Delivered on June 5, 2025",
        "654321": "Sennheiser Headphones - Delivered on June 25, 2025, 1-year warranty active",
        "789012": "Samsung Galaxy S23 - In transit, expected delivery on July 1, 2025",
    }
    
    return mock_orders.get(order_number, f"Order {order_number} not found. Please verify the order number.")

logger.info("✅ Check Order Status tool ready")

## Step 4: Initialize Session Manager

**NEW: This section introduces the MemorySessionManager for session-based Memory operations.**

In [ ]:
# Initialize the session memory manager
session_manager: MemorySessionManager = MemorySessionManager(memory_id=memory.id, region_name=REGION)

# Create a memory session for the specific customer
customer_session: MemorySession = session_manager.create_memory_session(
    actor_id=CUSTOMER_ID, 
    session_id=SESSION_ID
)

logger.info(f"✅ Session manager initialized for memory: {memory.id}")
logger.info(f"✅ Customer session created for actor: {CUSTOMER_ID}")
logger.info(f"   Session type: {type(customer_session)}")
logger.info(f"   Actor object: {customer_session.get_actor()}")

## Step 5: Create Memory Hook Provider for Customer Support
Hooks are special functions that run at specific points in an agent's execution lifecycle. Our custom hook provider will automatically manage customer support context by:
- **Saving support interactions** after each response using session-based methods
- **Retrieving and Injecting relevant context** from previous orders and preferences when processing new queries.


In [ ]:
class CustomerSupportMemoryHooks(HookProvider):
    """Memory hooks for customer support agent - ENHANCED with MemorySession"""
    
    def __init__(self, customer_session: MemorySession):
        # Accept MemorySession directly 
        self.customer_session = customer_session
        
        # Define retrieval configuration for different memory types
        self.retrieval_config = {
            "support/customer/{actorId}/preferences": RetrievalConfig(top_k=3, relevance_score=0.3),
            "support/customer/{actorId}/semantic": RetrievalConfig(top_k=5, relevance_score=0.2)
        }
    
    def retrieve_customer_context(self, event: MessageAddedEvent):
        """Retrieve customer context before processing support query using MemorySession"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_query = messages[-1]["content"][0]["text"]
            
            try:
                # Use MemorySession for context retrieval
                relevant_memories = []
                
                # Search across different memory namespaces using MemorySession
                for namespace_template, config in self.retrieval_config.items():
                    # Resolve namespace template with actual actor ID from session
                    resolved_namespace = namespace_template.format(
                        actorId=self.customer_session._actor_id
                    )
                    
                    # Use MemorySession API (no need to pass actor_id/session_id)
                    memories = self.customer_session.search_long_term_memories(
                        query=user_query,
                        namespace_prefix=resolved_namespace,
                        top_k=config.top_k
                    )
                    print(memories)
                    # Filter by relevance score
                    filtered_memories = [
                        memory for memory in memories
                        if memory.get("score", 0) >= config.relevance_score
                    ]
                    
                    relevant_memories.extend(filtered_memories)
                    logger.info(f"Found {len(filtered_memories)} relevant memories in {resolved_namespace}")
                
                # Inject context into agent's system prompt if memories found
                if relevant_memories:
                    context_text = self._format_context(relevant_memories)
                    original_prompt = event.agent.system_prompt
                    enhanced_prompt = f"{original_prompt}\n\nCustomer Context:\n{context_text}"
                    event.agent.system_prompt = enhanced_prompt
                    logger.info(f"✅ Injected {len(relevant_memories)} memories into agent context")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve customer context: {e}")
    
    def _format_context(self, memories: List[MemoryRecord]) -> str:
        """Format retrieved memories for agent context"""
        context_lines = []
        for i, memory in enumerate(memories[:5], 1):  # Limit to top 5
            content = memory.get('content', {}).get('text', 'No content available')
            score = memory.get('score', 0)
            context_lines.append(f"{i}. (Score: {score:.2f}) {content[:200]}...")
        
        return "\n".join(context_lines)
    
    def save_support_interaction(self, event: AfterInvocationEvent):
        """Save support interaction using MemorySession (cleaner API)"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last customer query and agent response
                customer_query = None
                agent_response = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not customer_query and "toolResult" not in msg["content"][0]:
                        customer_query = msg["content"][0]["text"]
                        break
                
                if customer_query and agent_response:
                    # Use MemorySession (no need to pass actor_id/session_id)
                    interaction_messages = [
                        ConversationalMessage(customer_query, USER),
                        ConversationalMessage(agent_response, ASSISTANT)
                    ]
                    
                    result = self.customer_session.add_turns(interaction_messages)
                    logger.info(f"✅ Saved interaction using MemorySession - Event ID: {result['eventId']}")
                    
        except Exception as e:
            logger.error(f"Failed to save support interaction: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register customer support memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_customer_context)  # Re-added!
        registry.add_callback(AfterInvocationEvent, self.save_support_interaction)
        logger.info("✅ Customer support memory hooks registered with MemorySession")
print('Executed!')

### Step 6: Create Customer Support Agent

In [ ]:
# Create memory hooks using MemorySession
support_hooks = CustomerSupportMemoryHooks(customer_session)

# Create customer support agent
support_agent = Agent(
    hooks=[support_hooks],
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[web_search, check_order_status],
    system_prompt="""You are a helpful customer support agent with access to customer history and order information. 
    
    Your role:
    - Help customers with their orders, returns, and product issues
    - Use customer context to provide personalized support
    - Search for product information when needed
    - Be empathetic and solution-focused
    - Reference previous orders and preferences when relevant
    
    Always be professional, helpful, and aim to resolve customer issues efficiently."""
)

print("✅ Customer support agent created with MemorySession integration")

### Step 7: Seed Customer History

Let's add some previous customer interactions to demonstrate memory functionality.

**NOTE: This section uses ConversationalMessage format and session-based storage.**

In [ ]:
# Seed with previous customer interactions using MemorySession
previous_interactions = [
    ConversationalMessage("I bought a new iPhone 15 Pro on June 1st, 2025. Order number is 123456.", USER),
    ConversationalMessage("Thank you for your purchase! I can see your iPhone 15 Pro order #123456 was delivered successfully. How can I help you today?", ASSISTANT),
    ConversationalMessage("I also ordered Sennheiser headphones on June 20th. Order number 654321. They came with 1-year warranty.", USER),
    ConversationalMessage("Perfect! I have your Sennheiser headphones order #654321 on file with the 1-year warranty. Both your iPhone and headphones should work great together.", ASSISTANT),
    ConversationalMessage("I'm looking for a good laptop. I prefer ThinkPad models.", USER),
    ConversationalMessage("Great choice! ThinkPads are excellent for their durability and performance. Let me help you find the right model for your needs.", ASSISTANT)
]

# Save using MemorySession 
try:
    event_response = customer_session.add_turns(previous_interactions)
    logger.info(f"✅ Seeded customer history using MemorySession")
    logger.info(f"   Event ID: {event_response['eventId']}")
except Exception as e:
    logger.error(f"⚠️ Error seeding history: {e}")


#### Agent is ready to go. 

### Lets test Customer Support Scenarios

In [ ]:
# Test 1: Customer reports iPhone issue 
logger.info("🧪 Running Test 1: iPhone performance issue")
test_query_1 = "My iPhone is running very slow and gets hot when charging. Can you help?"
logger.info(f"Query: {test_query_1}")

response1 = support_agent(test_query_1)
logger.info(f"✅ Test 1 completed successfully")
print(f"\n📱 iPhone Issue Support Response:\n{response1}\n")

In [ ]:
# Test 2: Bluetooth connectivity issue 
logger.info("🧪 Running Test 2: Bluetooth connectivity issue")
test_query_2 = "My iPhone won't connect to my Sennheiser headphones via Bluetooth. How do I fix this?"
logger.info(f"Query: {test_query_2}")

response2 = support_agent(test_query_2)
logger.info(f"✅ Test 2 completed successfully")
print(f"\n🎧 Bluetooth Issue Support Response:\n{response2}\n")

In [ ]:
# Test 3: Check order status 
logger.info("🧪 Running Test 3: Order status check")
test_query_3 = "Can you check the status of my recent orders?"
logger.info(f"Query: {test_query_3}")

response3 = support_agent(test_query_3)
logger.info(f"✅ Test 3 completed successfully")
print(f"\n📦 Order Status Support Response:\n{response3}\n")

In [ ]:
# Test 4: Product recommendation based on preferences 
logger.info("🧪 Running Test 4: Product recommendation")
test_query_4 = "I'm still interested in buying a laptop. What ThinkPad models do you recommend?"
logger.info(f"Query: {test_query_4}")

response4 = support_agent(test_query_4)
logger.info(f"✅ Test 4 completed successfully")
print(f"\n💻 Product Recommendation Support Response:\n{response4}\n")

logger.info("🎉 All customer support scenario tests completed!")

#### Customer Support Tutorial completed! 🎉 
Key takeaways:
- Memory hooks automatically manage customer context across support sessions using MemorySessionManager
- Multi-strategy memory captures orders, preferences, and facts from conversations using typed strategy classes
- Agents can provide personalized support based on customer history
- Tools can be integrated for order lookup and web search functionality
- Customer support becomes more efficient with persistent memory

## Clean Up

### Optional: Delete Memory Resource

In [ ]:
# Uncomment to delete the memory resource using MemoryManager
# try:
#     memory_manager.delete_memory(memory_id)
#     print(f"✅ Deleted memory resource using MemoryManager: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")